# Laserchicken experiment
This document shows how Laserchicken can be used to calculate features that describe a point cloud.

## Load a las file

In [5]:
from laserchicken import load

from laserchicken import compute_features
import requests

url = "http://172.17.0.4/testdata/AHN3.laz"
r = requests.get(url)
with open("/tmp/AHN3.laz", 'wb') as f:
    f.write(r.content)

point_cloud = load('/tmp/AHN3.laz')


from laserchicken.normalize import normalize
normalize(point_cloud)



from laserchicken.filter import select_polygon
polygon = "POLYGON(( 131963.984125 549718.375000," + \
                   " 132000.000125 549718.375000," + \
                   " 132000.000125 549797.063000," + \
                   " 131963.984125 549797.063000," + \
                   " 131963.984125 549718.375000))"
point_cloud = select_polygon(point_cloud, polygon)

# by attribute threshold
from laserchicken.filter import select_above, select_below
points_below_1_meter = select_below(point_cloud, 'normalized_height', 1)
points_above_1_meter = select_above(point_cloud, 'normalized_height', 1)


from laserchicken import compute_neighborhoods
from laserchicken import build_volume
targets = point_cloud
volume = build_volume('sphere', radius=5)
neighborhoods = compute_neighborhoods(point_cloud, targets, volume)

from laserchicken import compute_features
compute_features(point_cloud, neighborhoods, targets, ['std_z','mean_z','slope'], volume)

# We add 4 parameterized features.
from laserchicken import register_new_feature_extractor
from laserchicken.feature_extractor.band_ratio_feature_extractor import BandRatioFeatureExtractor
register_new_feature_extractor(BandRatioFeatureExtractor(None,1,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(1,2,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(2,None,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(None,0,data_key='z'))

# Print a list of all available feature names and notice the parameterized features we just registered, 'band_ratio_1<z<2'.
from laserchicken.feature_extractor.feature_extraction import list_feature_names
sorted(list_feature_names())

# Recalculate neighbors and calculate the parameterized feature
# Neighbors is a generator and can only be iterated once. We therefore recalculate them here. If we would want to do multiple calculations without recalculating the neighbors, we could copy the neighbors to a list. This is not done by default because neighbors can quickly grow quite large so that available RAM unnecessarily becomes the bottle neck.
cylinder = build_volume('infinite cylinder', radius=5)
neighborhoods = compute_neighborhoods(point_cloud, targets, cylinder)

point_cloud

Cylinder size in Bytes: 1262794583.0369534
Memory size in Bytes: 7779377152
Start tree creation
Done with env tree creation
Done with target tree creation
Extracting feature(s) "['eigenv_1', 'eigenv_2', 'eigenv_3', 'normal_vector_1', 'normal_vector_2', 'normal_vector_3', 'slope']"Extracting feature(s) "['eigenv_1', 'eigenv_2', 'eigenv_3', 'normal_vector_1', 'normal_vector_2', 'normal_vector_3', 'slope']" took 0.46 seconds
Extracting feature(s) "['mean_z', 'std_z', 'coeff_var_z']"Extracting feature(s) "['mean_z', 'std_z', 'coeff_var_z']" took 0.37 seconds
The following unrequested features were calculated as a side effect, but will not be returned: ['normal_vector_3', 'normal_vector_2', 'normal_vector_1', 'eigenv_3', 'eigenv_2', 'eigenv_1', 'coeff_var_z']


{'vertex': {'x': {'type': 'float64',
   'data': array([131997.625125, 131998.547125, 131999.687125, ..., 131999.469125,
          131999.687125, 131999.937125])},
  'y': {'type': 'float64',
   'data': array([549723.875, 549722.875, 549721.688, ..., 549795.563, 549795.313,
          549796.813])},
  'z': {'type': 'float64',
   'data': array([-0.26600002, -0.26000002, -0.21300002, ..., -0.14100002,
          -0.25600002, -0.37300002])},
  'raw_classification': {'type': 'uint8',
   'data': array([9, 9, 9, ..., 9, 9, 9], dtype=uint8)},
  'intensity': {'type': 'uint16',
   'data': array([152,  12,  16, ...,  14,  15, 591], dtype=uint16)},
  'gps_time': {'type': 'float64',
   'data': array([78563787.93570042, 78563787.93571067, 78563787.93572354, ...,
          78563786.99820375, 78563786.99820614, 78563786.97570777])},
  'normalized_height': {'type': 'float64',
   'data': array([1.311, 1.317, 1.364, ..., 1.436, 1.321, 1.204])},
  'slope': {'type': 'float64',
   'data': array([0.0089502 , 0.

## Normalize data
Adds the attribute 'normaiized_height' to the point cloud. Note that data is added to the given point cloud  in place 

In [4]:



point_cloud

Wall time: 185 ms


{'vertex': {'x': {'type': 'float64',
   'data': array([131999.984125, 131997.625125, 131998.547125, ..., 132501.531125,
          132501.391125, 132500.641125])},
  'y': {'type': 'float64',
   'data': array([549718.375, 549723.875, 549722.875, ..., 549966.313, 549964.875,
          549964.563])},
  'z': {'type': 'float64',
   'data': array([-0.34100002, -0.26600002, -0.26000002, ..., -0.24100002,
          -0.24100002, -0.24000002])},
  'raw_classification': {'type': 'uint8',
   'data': array([9, 9, 9, ..., 9, 9, 9], dtype=uint8)},
  'intensity': {'type': 'uint16',
   'data': array([ 41, 152,  12, ...,  10,  15,  10], dtype=uint16)},
  'gps_time': {'type': 'float64',
   'data': array([78563787.97322202, 78563787.93570042, 78563787.93571067, ...,
          78563778.28828931, 78563778.3107884 , 78563778.32578015])},
  'normalized_height': {'type': 'float64',
   'data': array([1.236, 1.311, 1.317, ..., 1.336, 1.336, 1.337])}},
 'log': [{'time': '2020-09-27 07:15:31.040881',
   'module': '

## Filter points 


In [5]:
# inside a polygon; We filter points within a polygon




Wall time: 1.4 s


## Compute neighbors
Computer neighbors for every point in the point cloud. This means that the target point cloud should contain the same points as the environment point cloud. The target point cloud contains the point at which the neighbors should be calculated.

The code below finds for every target point, the neighbors within a Spherical neighborhood with radius of 5 meters.

Wall time: 0 ns


## Calculate features

Cylinder size in Bytes: 1214225560.6124551
Memory size in Bytes: 8271544320
Start tree creation
Done with env tree creation
Done with target tree creation
Extracting feature(s) "['eigenv_1', 'eigenv_2', 'eigenv_3', 'normal_vector_1', 'normal_vector_2', 'normal_vector_3', 'slope']"Extracting feature(s) "['eigenv_1', 'eigenv_2', 'eigenv_3', 'normal_vector_1', 'normal_vector_2', 'normal_vector_3', 'slope']" took 2.67 seconds
Extracting feature(s) "['mean_z', 'std_z', 'coeff_var_z']"Extracting feature(s) "['mean_z', 'std_z', 'coeff_var_z']" took 0.38 seconds
The following unrequested features were calculated as a side effect, but will not be returned: ['normal_vector_3', 'normal_vector_2', 'normal_vector_1', 'eigenv_3', 'eigenv_2', 'eigenv_1', 'coeff_var_z']
Cylinder size in Bytes: 1214225560.6124551
Memory size in Bytes: 8271544320
Start tree creation
Done with env tree creation
Done with target tree creation
Extracting feature(s) "['band_ratio_1<normalized_height<2']"Extracting feature(s

## Write result to ply file

In [8]:
from laserchicken import export
export(point_cloud, 'my_output.ply')

Wall time: 394 ms


In [9]:
# import matplotlib.pyplot as plt